In [ ]:
# Author: Aaron Valoroso
# Date: June 13th, 2018
# Topic: WMTS and image stitching

from PIL import Image #conda install -c anaconda pillow 
import numpy as np
import shutil
import os
import sys
import math
from itertools import product
from ipywidgets import widgets, HBox, Label# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

In [ ]:
import requests
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

def download(destination, url, filename):
    temp_path = None
    if destination[-1] != '/':
        destination += '/'

    if os.path.exists(destination):
        the_sesh = requests.Session()
        retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])
        the_sesh.mount('http://', HTTPAdapter(max_retries=retries))
        response = the_sesh.get(url, verify=False)
        temp_path = destination + filename
        if response.status_code == 200:
            open(temp_path, 'wb').write(response.content)
    else:
        raise ValueError("Sorry, but the download destination that you requested is not a path.")

In [ ]:
def image_stitch(current_path, max_width, max_height, min_width, min_height, tile_width_size, tile_height_size, images):
    # Create the new image with the incoming max width and height.
    stitched_image = Image.new('RGB', (max_width, max_height))
    
    for file in images:
        incoming_image = Image.open(current_path + file)
        filename, file_extension = os.path.splitext(file)
        x, y = filename.split('-')
        x = (int(x) - min_width) * tile_width_size
        y = (int(y) - min_height) * tile_height_size
        stitched_image.paste(im=incoming_image, box=(x, y))
        incoming_image.close()

    return stitched_image

In [ ]:
# Lets see if the user wants to download any data first.
print("Downloading Data True/False: ")
download_widget_value = widgets.Checkbox(value=False)
display(download_widget_value)

In [ ]:
# Get input data
my_switch = download_widget_value.value

style = {'description_width': 'initial'}

if my_switch == True:
    zoom_widget_value = widgets.IntSlider(min=0,max=19,step=1,description='Zoom Level:',orientation='horizontal')
    display(zoom_widget_value)
    
    print("Upper left hand corner of bbox.")
    long_1_widget_value = widgets.FloatText(description='Longitude-1:')
    display(long_1_widget_value)

    lat_1_widget_value = widgets.FloatText(description='Latitude-1:')
    display(lat_1_widget_value)
    
    print("Bottom right hand corner of bbox.")
    long_2_widget_value = widgets.FloatText(description='Longitude-2:')
    display(long_2_widget_value)

    lat_2_widget_value = widgets.FloatText(description='Latitude-2:')
    display(lat_2_widget_value)
    
    download_location_widget_value = widgets.Text(description="Download Location / Name: ", style=style)
    hbox_value = HBox([download_location_widget_value])
    display(hbox_value)
else:
    import_location_widget_value = widgets.Text(description="Import Folder Location: ", style=style)
    hbox_value = HBox([import_location_widget_value])    
    display(hbox_value)

In [ ]:
# Main
%time
# Checking the input.
current_path = os.getcwd()
images = list()
if my_switch == True:
    zoom_level = zoom_widget_value.value
    long_1 = long_1_widget_value.value
    lat_1 = lat_1_widget_value.value
    long_2 = long_2_widget_value.value
    lat_2 = lat_2_widget_value.value
#     zoom_level = 11
#     long_1 = -98.136383
#     lat_1 = 30.656413
#     long_2 = -97.574363
#     lat_2 = 30.004787
    url = 'http://c.tile.openstreetmap.org/{0}/{1}/{2}.png'
    
    # Make sure none of the coordinates are not zero. This may need to be changed because there might
    # be a coordinate that has a zero in it.
    check_coordinates = [long_1, long_2, lat_1, lat_2]
    for coor in check_coordinates:
        if coor == 0:
            sys.exit("There was a zero found in one of your coordinates...!")
    
    download_destination = download_location_widget_value.value
    current_path = current_path + '/' + download_destination
    if not os.path.exists(current_path):
        os.makedirs(current_path)
    else:
        print("We are deleteing the current download destination...!")
        shutil.rmtree(current_path)
        os.makedirs(current_path)

    # Get the tile number for the latitude of both the upper left and bottom right
    # hand corner of the bbox.
    the_lats = [lat_1, lat_2]
    print("Lat: ", the_lats)
    the_lats = [int(1/(2*np.pi)*2**zoom_level*(np.pi-np.log(np.tan(np.pi/4+np.radians(y)/2)))) for y in the_lats]

    # Get the tile number for the longitude of both the upper left and bottom right
    # hand corner of the bbox.
    the_longs = [long_1, long_2]
    print("Long: ", the_longs)
    the_longs = [int(2**(zoom_level-1)*(x/180+1)) for x in the_longs]
    
    print("Conv-Lat: ", the_lats)
    print("Conv-Long: ", the_longs)
    
    z = 1
    first_lat = math.degrees(2*(math.atan(math.exp(math.pi-(the_longs[0]*2*math.pi)/2**z))-math.pi/4))
    second_lat = math.degrees(2*(math.atan(math.exp(math.pi-((the_longs[-1] + 1)*2*math.pi)/2**z))-math.pi/4))
    
    first_long = lon = math.degrees(the_lats[0]*2*math.pi/(2**z) - math.pi)
    second_long = lon = math.degrees((the_lats[-1] + 1)*2*math.pi/(2**z) - math.pi)
    
    # Get the range of tiles
    lat_range = range(the_lats[0], (the_lats[1] + 1))
    long_range = range(the_longs[0], (the_longs[1] + 1))
    print("Lat-R: ", lat_range)
    print("Long-R: ", long_range)
    # Download all the tiles and get the upper right and bottom left tile coordinate.
    loop_itereator = 0
    for x, y in product(long_range, lat_range):
        filename = "{0}-{1}.png".format(x, y)
        download(current_path, url.format(zoom_level, x, y), filename)  
        images.append(filename)
        if loop_itereator == 0:
            test_image = Image.open(current_path + '/' + filename)
            (tile_width_size, tile_height_size) = test_image.size
            test_image.close()
            min_height = y
            min_width = x
            loop_itereator += 1
    
    # Subtract the max from the min to get the height and width, add one because we are starting at zero, and 
    # then multiply by the max and width of the title. (Calculating the size of the image.)
    max_width = ((x - min_width) + 1) * tile_width_size
    max_height = ((y - min_height) + 1) * tile_height_size
else:
    import_destination = import_location_widget_value.value
    current_path = current_path + '/' + import_destination
    if os.path.exists(current_path):
        temp_store_find = current_path + '/' + '.DS_Store'
        if os.path.exists(temp_store_find):
            os.remove(temp_store_find)
        
        max_height = 0
        max_width = 0
        loop_iterator = 0
        for file in os.listdir(current_path):
            images.append(file)
            filename, file_extension = os.path.splitext(file)
            number_1, number_2 = filename.split('-')
            number_1 = int(number_1)
            number_2 = int(number_2)
            if loop_iterator == 0:
                test_image = Image.open(current_path + '/'  + file)
                (tile_width_size, tile_height_size) = test_image.size
                test_image.close()
                min_width = number_1
                min_height = number_2
                loop_iterator += 1
            else:
                if number_1 > max_width:
                    max_width = number_1
                if number_2 > max_height:
                    max_height = number_2
                if number_1 < min_width:
                    min_width = number_1
                if number_2 < min_height:
                    min_height = number_2
        
        max_height = ((max_height - min_height) + 1) * tile_height_size
        max_width = ((max_width - min_width) + 1) * tile_width_size
    else:
        sys.exit("Unable to find your import folder...!")

current_path += '/'
# Print out the number of colums and rows that we have for our bbox.
print("The number of columns: " + str(max_width))
print("The number of rows: " + str(max_height))

# Put our images together.
my_image = image_stitch(current_path, max_width, max_height, min_width, min_height, tile_width_size, tile_height_size, images)
my_image.save(current_path + 'Thy-Picture.png')
my_image.show()
my_image.close()